In [1]:
# coding: utf-8

import pandas as pd

from files.base import Base

df = pd.read_csv(Base.FILE_FINAL, sep=";")
df.head(2)


,id,Titulo,Manual,Bidirecional Naive,Bidirecional Senti,Naive Bayes,SentiStregth,Positivo,Negativo,Neutro,Link,Origem,tamanho,palavras neutras,caracteres
0,1,"Doria chama gestão da pandemia de 'descaso', e...",NaN,partial,partial,partial,partial,2,-3,-1,https://diariodonordeste.verdesmares.com.br/po...,Diário do Nordeste,25,4,2449
1,2,Câmara aprova suspensão da carteira do motoris...,NaN,partial,partial,partial,partial,3,-3,-1,https://g1.globo.com/politica/noticia/2021/09/...,G1 Ceará,24,4,3435


In [3]:
dataFrameAnalytics = pd.DataFrame(columns=[
    'model',
    'accuracy',
    'recall_impartial',
    'precision_imparcial',
    'recall_partial',
    'precision_parcial',
    'TP_impartial',
    'FP_impartial',
    'FN_impartial',
    'TN_impartial',
    'TP_partial',
    'FP_partial',
    'FN_partial',
    'TN_partial',
])

def metric_model(key):
    manual = df['Manual'].value_counts()
    hit = df[df[key] == df['Manual']][key].value_counts()
    
    TP_impartial = hit.impartial
    FP_impartial = manual.partial - hit.partial
    FN_impartial = manual.impartial - hit.impartial
    TN_impartial = hit.partial

    TP_partial = hit.partial
    FP_partial = manual.impartial - hit.impartial
    FN_partial = manual.partial - hit.partial
    TN_partial = hit.impartial

    accuracy = (TP_impartial + TP_partial) / (manual.impartial + manual.partial)

    recall_impartial = TP_impartial / (TP_impartial + FN_impartial)
    precision_imparcial = TP_impartial / (TP_impartial + FP_impartial)

    recall_partial = TP_partial / (TP_partial + FN_partial)
    precision_parcial = TP_partial / (TP_partial + FP_partial)

    return {
        'model': key,
        'accuracy': accuracy,
        'recall_impartial': recall_impartial,
        'precision_imparcial': precision_imparcial,
        'recall_partial': recall_partial,
        'precision_parcial': precision_parcial,
        'TP_impartial': TP_impartial,
        'FP_impartial': FP_impartial,
        'FN_impartial': FN_impartial,
        'TN_impartial': TN_impartial,
        'TP_partial': TP_partial,
        'FP_partial': FP_partial,
        'FN_partial': FN_partial,
        'TN_partial': TN_partial
    }

dataFrameAnalytics = dataFrameAnalytics.append(metric_model('Bidirecional Naive'), ignore_index=True)
dataFrameAnalytics = dataFrameAnalytics.append(metric_model('Bidirecional Senti'), ignore_index=True)
dataFrameAnalytics = dataFrameAnalytics.append(metric_model('Naive Bayes'), ignore_index=True)
dataFrameAnalytics = dataFrameAnalytics.append(metric_model('SentiStregth'), ignore_index=True)

analyticsView = pd.DataFrame(columns=[
    'metric',
    'Bidirecional Naive',
    'Bidirecional Senti',
    'Naive Bayes',
    'SentiStregth',
])

def metric_view(key):
    return {
        'metric': key,
        'Bidirecional Naive': str(round(dataFrameAnalytics[dataFrameAnalytics['model'] == 'Bidirecional Naive'][key].values[0] * 100, 2)) + ' %',
        'Bidirecional Senti': str(round(dataFrameAnalytics[dataFrameAnalytics['model'] == 'Bidirecional Senti'][key].values[0] * 100, 2)) + ' %',
        'Naive Bayes': str(round(dataFrameAnalytics[dataFrameAnalytics['model'] == 'Naive Bayes'][key].values[0] * 100, 2)) + ' %',
        'SentiStregth': str(round(dataFrameAnalytics[dataFrameAnalytics['model'] == 'SentiStregth'][key].values[0] * 100, 2)) + ' %'
    }

analyticsView = analyticsView.append(metric_view('accuracy'), ignore_index=True)
analyticsView = analyticsView.append(metric_view('recall_impartial'), ignore_index=True)
analyticsView = analyticsView.append(metric_view('precision_imparcial'), ignore_index=True)
analyticsView = analyticsView.append(metric_view('recall_partial'), ignore_index=True)
analyticsView = analyticsView.append(metric_view('precision_parcial'), ignore_index=True)

analyticsView


,metric,Bidirecional Naive,Bidirecional Senti,Naive Bayes,SentiStregth
0,accuracy,73.67 %,68.33 %,73.33 %,57.33 %
1,recall_impartial,80.6 %,69.83 %,84.91 %,57.33 %
2,precision_imparcial,84.62 %,86.63 %,81.4 %,82.1 %
3,recall_partial,50.0 %,63.24 %,33.82 %,57.35 %
4,precision_parcial,43.04 %,38.05 %,39.66 %,28.26 %


In [4]:
dataFrameAnalytics = pd.DataFrame(columns=[
    '',
    'Total',
    'Imparcial',
    'Parcial'
])

manual = df['Manual'].value_counts()
b_naive = df['Bidirecional Naive'].value_counts()
b_senti = df['Bidirecional Senti'].value_counts()
naive = df['Naive Bayes'].value_counts()
senti = df['SentiStregth'].value_counts()

dataFrameAnalytics = dataFrameAnalytics.append({
    '': 'Manual',
    'Total': (manual.impartial + manual.partial),
    'Imparcial': manual.impartial,
    'Parcial': manual.partial
}, ignore_index=True)

dataFrameAnalytics = dataFrameAnalytics.append({
    '': 'Bidirecional Naive',
    'Total': (b_naive.impartial + b_naive.partial),
    'Imparcial': b_naive.impartial,
    'Parcial': b_naive.partial
}, ignore_index=True)

dataFrameAnalytics = dataFrameAnalytics.append({
    '': 'Bidirecional Senti',
    'Total': (b_senti.impartial + b_senti.partial),
    'Imparcial': b_senti.impartial,
    'Parcial': b_senti.partial
}, ignore_index=True)

dataFrameAnalytics = dataFrameAnalytics.append({
    '': 'Naive Bayes',
    'Total': (naive.impartial + naive.partial),
    'Imparcial': naive.impartial,
    'Parcial': naive.partial
}, ignore_index=True)

dataFrameAnalytics = dataFrameAnalytics.append({
    '': 'SentiStregth',
    'Total': (senti.impartial + senti.partial),
    'Imparcial': senti.impartial,
    'Parcial': senti.partial
}, ignore_index=True)

dataFrameAnalytics

,,Total,Imparcial,Parcial
0,Manual,300,232,68
1,Bidirecional Naive,18175,9731,8444
2,Bidirecional Senti,18175,7598,10577
3,Naive Bayes,18175,10344,7831
4,SentiStregth,18175,8501,9674


In [5]:
total = df['id'].count()
totalDN = df[(df['Origem'] == "Diário do Nordeste")]['id'].count()
totalG1 = df[(df['Origem'] == "G1 Ceará")]['id'].count()
totalOP = df[(df['Origem'] == "O Povo")]['id'].count()

impartial = df[df['Bidirecional Naive'] == "impartial"]['id'].count()
pImpartial = impartial / total * 100

impartialDN = df[(df['Origem'] == "Diário do Nordeste") & (df['Bidirecional Naive'] == "impartial")]['id'].count()
pImpartialDN = impartialDN / totalDN * 100

impartialG1 = df[(df['Origem'] == "G1 Ceará") & (df['Bidirecional Naive'] == "impartial")]['id'].count()
pImpartialG1 = impartialG1 / totalG1 * 100

impartialOP = df[(df['Origem'] == "O Povo") & (df['Bidirecional Naive'] == "impartial")]['id'].count()
pImpartialOP = impartialOP / totalOP * 100

partial = df[df['Bidirecional Naive'] == "partial"]['id'].count()
pPartial = partial / total * 100

partialDN = df[(df['Origem'] == "Diário do Nordeste") & (df['Bidirecional Naive'] == "partial")]['id'].count()
pPartialDN = partialDN / totalDN * 100

partialG1 = df[(df['Origem'] == "G1 Ceará") & (df['Bidirecional Naive'] == "partial")]['id'].count()
pPartialG1 = partialG1 / totalG1 * 100

partialOP = df[(df['Origem'] == "O Povo") & (df['Bidirecional Naive'] == "partial")]['id'].count()
pPartialOP = partialOP / totalOP * 100

analytics = pd.DataFrame({
    'total' : {
        'Total': total,
        'Diário do Nordeste': totalDN,
        'G1 Ceará': totalG1,
        'O Povo': totalOP
    },
    'impartial' : {
        'Total': str(impartial) + " ( " + str(round(pImpartial)) + "% )",
        'Diário do Nordeste': str(impartialDN) + " ( " + str(round(pImpartialDN)) + "% )",
        'G1 Ceará': str(impartialG1) + " ( " + str(round(pImpartialG1)) + "% )",
        'O Povo': str(impartialOP) + " ( " + str(round(pImpartialOP)) + "% )",
    },
    'partial' : {
        'Total':  str(partial) + " ( " + str(round(pPartial)) + "% )",
        'Diário do Nordeste': str(partialDN) + " ( " + str(round(pPartialDN)) + "% )",
        'G1 Ceará': str(partialG1) + " ( " + str(round(pPartialG1)) + "% )",
        'O Povo': str(partialOP) + " ( " + str(round(pPartialOP)) + "% )",
    },
})

analytics

,total,impartial,partial
Total,18175,9731 ( 54% ),8444 ( 46% )
Diário do Nordeste,2570,728 ( 28% ),1842 ( 72% )
G1 Ceará,7101,3286 ( 46% ),3815 ( 54% )
O Povo,8504,5717 ( 67% ),2787 ( 33% )
